NLP Processing

In [5]:
!pip install --upgrade pip
!pip install re
!pip install typer
!pip install spacy
!pip install scispacy

SyntaxError: invalid syntax (1389288435.py, line 1)

In [36]:
nlp = spacy.load("en_core_web_sm")


In [ ]:
disease_chems=pd.read_csv('/home/giannis-charalampopoulos/M2PHDS/web based data/wbd-project/CTD_disease_chems.csv')

In [ ]:
pmid_list=disease_chems['PubMedIDs'].tolist()

In [38]:
import requests
import time
import pandas as pd
from xml.etree import ElementTree as ET

# Your list of PMIDs
pmid_list = list(set(pmid_list))  # remove duplicates
total = len(pmid_list)

def fetch_pubmed_xml(pmid):
    """Fetch a single PubMed record via NCBI EFetch."""
    url = (
        "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        f"?db=pubmed&id={pmid}&retmode=xml"
    )

    response = requests.get(url)
    if response.status_code != 200:
        print(f"⚠️ Failed to fetch {pmid}")
        return None

    return response.text

records = []
for idx, pmid in enumerate(pmid_list, start=1):

    xml_data = fetch_pubmed_xml(pmid)
    if xml_data is None:
        print(f"❌ Could not retrieve PMID {pmid} ({idx}/{total})")
        continue

    # Parse XML
    root = ET.fromstring(xml_data)
    title = root.findtext(".//ArticleTitle")
    abstract = " ".join(
        [t.text for t in root.findall(".//AbstractText") if t.text]
    )

    records.append({
        "pmid": pmid,
        "title": title,
        "abstract": abstract,
        "xml_raw": xml_data
    })

    # Print progress for every PMID found
    print(f"✔️  Retrieved {idx}/{total} → PMID {pmid}")

    # Optional milestone print every 25 items
    if idx % 25 == 0:
        print(f"⏳ Progress: {idx}/{total} ({100*idx/total:.1f}%)")

    # polite delay (NCBI recommends <10 req/sec)
    time.sleep(0.15)

print("DONE! All available PMIDs retrieved.")
df = pd.DataFrame(records)
print("Total records downloaded:", len(df))

df.head()


✔️  Retrieved 1/345 → PMID 14755447
✔️  Retrieved 2/345 → PMID 19242545
✔️  Retrieved 3/345 → PMID 19921286
✔️  Retrieved 4/345 → PMID 21572417
✔️  Retrieved 5/345 → PMID 15108192|17033973
✔️  Retrieved 6/345 → PMID 17028446
✔️  Retrieved 7/345 → PMID 28316773
✔️  Retrieved 8/345 → PMID 17106428|18664314
✔️  Retrieved 9/345 → PMID 15892143
✔️  Retrieved 10/345 → PMID 8964908
✔️  Retrieved 11/345 → PMID 17349978|20662941
✔️  Retrieved 12/345 → PMID 21398041
✔️  Retrieved 13/345 → PMID 19365831
✔️  Retrieved 14/345 → PMID 20981775
✔️  Retrieved 15/345 → PMID 2606882|3709993|6502317
✔️  Retrieved 16/345 → PMID 19540081
✔️  Retrieved 17/345 → PMID 19404256
✔️  Retrieved 18/345 → PMID 17883863
✔️  Retrieved 19/345 → PMID 20952458
✔️  Retrieved 20/345 → PMID 37047547|38679149
✔️  Retrieved 21/345 → PMID 14585753
✔️  Retrieved 22/345 → PMID 20468076
✔️  Retrieved 23/345 → PMID 16904652
✔️  Retrieved 24/345 → PMID 30894343|35802248
✔️  Retrieved 25/345 → PMID 19401682
⏳ Progress: 25/345 (7.2%)

,pmid,title,abstract,xml_raw
0,14755447,"Association of tryptophan 2,3 dioxygenase gene...",Although elevation of blood and platelet serot...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
1,19242545,Association and mutation analyses of 16p11.2 a...,Autism is a complex childhood neurodevelopment...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
2,19921286,Novel variants identified in methyl-CpG-bindin...,Misregulation of the methyl-CpG-binding protei...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
3,21572417,Exome sequencing in sporadic autism spectrum d...,Evidence for the etiology of autism spectrum d...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
4,15108192|17033973,Association between the FOXP2 gene and autisti...,Several genomewide screens indicated that chro...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."


In [39]:
pubmed_dataset=df

In [40]:
pubmed_dataset=pubmed_dataset[['pmid', 'abstract']]
pubmed_dataset



,pmid,abstract
0,14755447,Although elevation of blood and platelet serot...
1,19242545,Autism is a complex childhood neurodevelopment...
2,19921286,Misregulation of the methyl-CpG-binding protei...
3,21572417,Evidence for the etiology of autism spectrum d...
4,15108192|17033973,Several genomewide screens indicated that chro...
...,...,...
339,17322880|18057082|18621663|20157312,"Autism spectrum disorders (ASDs) are common, h..."
340,20678249,Autism spectrum disorders (ASDs) are a group o...
341,6150139,Succinyladenosine and succinylaminoimidazole c...
342,17497416|24676906,"This study determined the level of mercury, le..."


In [41]:
pubmed_dataset = pubmed_dataset[
    pubmed_dataset["abstract"].astype(str).str.strip() != ""
]


In [42]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

# Downloads
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()

# Function to convert nltk pos tags to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize + POS tag + Lemmatize
pubmed_dataset["tokens"] = (
    pubmed_dataset["abstract"]
        .astype(str)
        .str.lower()
        .apply(word_tokenize)
        .apply(lambda words: pos_tag(words))
        .apply(lambda tagged: [
            lemmatizer.lemmatize(w, get_wordnet_pos(tag))
            for w, tag in tagged
        ])
)


[nltk_data] Downloading package punkt to /home/giannis-
[nltk_data]     charalampopoulos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/giannis-charalampopoulos/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/giannis-
[nltk_data]     charalampopoulos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/tmp/ipykernel_6320/988891669.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed_dataset["tokens"] = (


In [43]:
import re

def custom_tokenize(text):
    # keep numbers like 15,000 or 3.5e10 as one token
    pattern = r'\d[\d,\.]*|\w+'
    return re.findall(pattern, text.lower())


In [44]:

pubmed_dataset["tokens"]

0      [although, elevation, of, blood, and, platelet...
1      [autism, be, a, complex, childhood, neurodevel...
2      [misregulation, of, the, methyl-cpg-binding, p...
3      [evidence, for, the, etiology, of, autism, spe...
4      [several, genomewide, screen, indicate, that, ...
                             ...                        
339    [autism, spectrum, disorder, (, asds, ), be, c...
340    [autism, spectrum, disorder, (, asds, ), be, a...
341    [succinyladenosine, and, succinylaminoimidazol...
342    [this, study, determine, the, level, of, mercu...
343    [autism, spectrum, disorder, (, asds, ), be, a...
Name: tokens, Length: 341, dtype: object

In [45]:
pubmed_dataset["tokens"] = pubmed_dataset["tokens"].apply(
    lambda lst: [t for t in lst if any(c.isalnum() for c in t)]
)

/tmp/ipykernel_6320/3246752615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed_dataset["tokens"] = pubmed_dataset["tokens"].apply(


In [46]:
pubmed_dataset=pubmed_dataset.rename(columns={"tokens":"Text"})

In [47]:
pubmed_dataset_processed=pubmed_dataset[["pmid", "Text"]]


In [48]:
merged = pd.merge(
    pubmed_dataset_processed,
    disease_chems,
    left_on="pmid",
    right_on="PubMedIDs",
    how="inner"
)


In [49]:
RELATION_PATTERNS = [
    # --- Causation ---
    "cause", "causes", "caused", "causing",
    "induce", "induces", "induced", "inducing",
    "trigger", "triggers", "triggered",
    "provoke", "provokes", "provoked",
    "lead to", "leading to", "led to",
    "result in", "resulting in",
    "give rise to", "give rise",
    "bring about",
    "responsible for",
    "due to",
    "attributable to",

    # --- Association / correlation ---
    "associate", "associated", "associates", "associating",
    "correlate", "correlated", "correlates",
    "linked to", "link to", "linked with",
    "relationship with",
    "related to", "relation with",
    "connection with", "connected to",

    # --- Increase / decrease effects ---
    "increase", "increases", "increased",
    "elevate", "elevates", "elevated",
    "raise", "raises", "raised",
    "promote", "promotes", "promoted",
    "enhance", "enhances", "enhanced",
    "upregulate", "upregulated", "up-regulate",
    "stimulate", "stimulates", "stimulated",

    "decrease", "decreases", "decreased",
    "reduce", "reduces", "reduced",
    "lower", "lowers", "lowered",
    "suppress", "suppresses", "suppressed",
    "inhibit", "inhibits", "inhibited",
    "downregulate", "downregulated", "down-regulate",

    # --- Risk / probability ---
    "risk of", "increase risk", "reduced risk",
    "risk factor", "risk factors",
    "predispose", "predisposes", "predisposed",

    # --- Toxicity / harmful effects ---
    "toxic to", "toxicity",
    "harmful to",
    "adverse effect", "adverse effects",
    "side effect", "side effects",
    "damage", "damages", "damaged",

    # --- Protective associations ---
    "protective against",
    "protection against",
    "protects against",
    "prevents", "prevent", "prevented",
    "shield against",

    # --- Mechanistic biology ---
    "modulate", "modulates", "modulated",
    "regulate", "regulates", "regulated",
    "mediate", "mediates", "mediated",
    "mechanism of",
    "pathway",
    "involved in",
    "contribute to", "contributes to", "contributed to",

    # --- Reporting relationships ---
    "reported in",
    "observed in",
    "identified as",
    "detected in",
    "found in",
    "noted in",

    # --- Biomed phrasing ---
    "associated with higher levels",
    "associated with lower levels",
    "marker of",
    "indicator of",
    "linked to increased",
    "linked to decreased"
]


In [50]:
import pandas as pd
from collections import Counter
import math

final_dataset = merged  # your dataframe

# Pre-defined association patterns (expandable)


def match_pattern(tokens):
    text = " ".join(tokens)
    hits = 0
    for p in PATTERNS:
        if " ".join(p) in text:
            hits += 1
    return hits

total = len(final_dataset)

pair_counts = Counter()
pair_pattern_hits = Counter()

for i, row in final_dataset.iterrows():

    if i % 5000 == 0:
        print(f"{i}/{total}")

    chem = row["ChemicalName"]
    dis  = row["DiseaseName"]
    tokens = row["Text"]  # already a list of words

    pair = (chem, dis)

    # Count co-occurrence (each row is one occurrence)
    pair_counts[pair] += 1

    # Count pattern evidence
    pair_pattern_hits[pair] += match_pattern(tokens)

# Build results table
rows = []
for (chem, dis), count in pair_counts.items():
    pattern = pair_pattern_hits[(chem, dis)]
    pmi_like = math.log((pattern + 1) / (count + 1))  # pseudo-PMI for scoring

    rows.append({
        "ChemicalName": chem,
        "DiseaseName": dis,
        "CoOccurrences": count,
        "PatternEvidence": pattern,
        "EvidenceScore": pmi_like
    })

result = pd.DataFrame(rows).sort_values("EvidenceScore", ascending=False)


0/109192
5000/109192
10000/109192
15000/109192
20000/109192
25000/109192
30000/109192
35000/109192
40000/109192
45000/109192
50000/109192
55000/109192
60000/109192
65000/109192
70000/109192
75000/109192
80000/109192
85000/109192
90000/109192
95000/109192
100000/109192
105000/109192


In [51]:
result

,ChemicalName,DiseaseName,CoOccurrences,PatternEvidence,EvidenceScore
2530,S-4-bromobenzylglutathione cyclopentyl diester,Autistic Disorder,2,8,1.098612
2046,"Lys49-phospholipase A2, Bothrops lternatus",Autistic Disorder,2,8,1.098612
2857,zomepirac,Autistic Disorder,2,8,1.098612
1873,Hydrocarbons,Autistic Disorder,3,10,1.011601
1758,gentian root extract,Autistic Disorder,2,7,0.980829
...,...,...,...,...,...
8189,Glycocholic Acid,Autism Spectrum Disorder,43,0,-3.784190
7522,Vancomycin,Autism Spectrum Disorder,44,0,-3.806662
8188,Glycochenodeoxycholic Acid,Autism Spectrum Disorder,45,0,-3.828641
8171,Deoxycholic Acid,Autism Spectrum Disorder,48,0,-3.891820
